In [17]:
import pandas as pd
import nltk
import re
import string
from nltk import word_tokenize
#nltk.download('stopwords')
#nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [42]:
#---------------STARTING WITH R8----------------------------
##------------BUILDING THE DATASET
X_train = pd.read_csv('datasets/r8-train-all-terms.txt', sep="\t", header=None)
X_test = pd.read_csv('datasets/r8-test-all-terms.txt', sep="\t", header=None)
data_r8=pd.concat([X_train,X_test], ignore_index=True)
data_r8.columns = ["class", "text"]
classes_count = data_r8.groupby('class').count().sort_values(by=['text'],ascending=False)
classes_count

,text
class,
earn,3923
acq,2292
crude,374
trade,326
money-fx,293
interest,271
ship,144
grain,51


In [18]:
##------------NLP PRE-PROCESSING START FROM HERE-------------##
def nlp_preprocessing(text):
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # Replace nweline by some space
    text = text.replace('\r\n', ' ').replace('\n', ' ')
    word_tokens = word_tokenize(text)  # n_rows 1971
    stems = ''
    for word in word_tokens:
        stemed_word = stemmer.stem(word)
        if ((stemed_word not in stopwords) and (re.search('[a-zA-Z]', stemed_word)) and stemed_word.isalpha() and len(stemed_word) > 3):
            stems = stems + ' ' + stemed_word

    return stems[1:]  # to remove the first space of the file

In [38]:
data_r8_processed = data_r8
for index ,row in data_r8_processed.iterrows():
    print(index, end="\r")
    row['text'] = nlp_preprocessing(row['text'])

In [54]:
len(nlp_preprocessing(data_r8.iloc[5]["text"]))
len(data_r8.iloc[5]["text"])

1346

In [52]:
print(len(data_r8_processed.iloc[5]["text"]))
print(len(data_r8.iloc[5]["text"]))

853
1346
